In [8]:
import pandas as pd

In [9]:
import datetime as datetime

In [10]:
access_path = '../../data/processed/'
bal_file = 'bal_btw_rebal.dat'
# bal_file = 'balance_rebaldate_0930_all.dat'

#### 매일의 잔고(detail 테이블) INSERT

In [11]:
df_bal = pd.read_csv(access_path+bal_file, parse_dates=['date'])

In [12]:
df_bal.loc[(df_bal.date>='2021-10-29') & (df_bal.userid=='A009'), :]

,date,itemcode,userid,name,asset_class,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,original
1649,2021-10-29,A237370,A009,적극투자형대규모,Alternative,KODEX 배당성장채권혼합,25.0,12160.0,304000.0,12035.0,300875.0,0.237141,NaN,NaN
1650,2021-10-29,A157490,A009,적극투자형대규모,Equity,TIGER 소프트웨어,19.0,16260.0,308940.0,17140.0,325660.0,0.256676,NaN,NaN
1651,2021-10-29,A278540,A009,적극투자형대규모,Equity,KODEX MSCI Korea TR,23.0,12875.0,296125.0,12595.0,289685.0,0.228321,NaN,NaN
1652,2021-10-29,A102780,A009,적극투자형대규모,Equity,KODEX 삼성그룹,17.0,10075.0,171275.0,9970.0,169490.0,0.133587,NaN,NaN
1653,2021-10-29,A069500,A009,적극투자형대규모,Equity,KODEX 200,3.0,40195.0,120585.0,39460.0,118380.0,0.093304,NaN,NaN
1654,2021-10-29,A114260,A009,적극투자형대규모,Fixed Income,KODEX 국고채3년,1.0,57760.0,57760.0,57210.0,57210.0,0.045091,NaN,NaN
1655,2021-10-29,C000001,A009,적극투자형대규모,Cash,현금,7460.0,1.0,7460.0,1.0,7460.0,0.005880,NaN,NaN
1721,2021-10-30,A237370,A009,적극투자형대규모,Alternative,KODEX 배당성장채권혼합,25.0,12160.0,304000.0,12035.0,300875.0,0.237141,NaN,NaN
1722,2021-10-30,A157490,A009,적극투자형대규모,Equity,TIGER 소프트웨어,19.0,16260.0,308940.0,17140.0,325660.0,0.256676,NaN,NaN
1723,2021-10-30,A278540,A009,적극투자형대규모,Equity,KODEX MSCI Korea TR,23.0,12875.0,296125.0,12595.0,289685.0,0.228321,NaN,NaN


In [13]:
df_bal.date = df_bal['date'].dt.strftime('%#m/%#d/%Y 4:%M:%S PM')
# df_bal.date = df_bal['date'].dt.strftime('%-m/%-d/%Y 4:%M:%S PM')

In [14]:
df_bal.date

0        10/7/2021 4:00:00 PM
1        10/7/2021 4:00:00 PM
2        10/7/2021 4:00:00 PM
3        10/7/2021 4:00:00 PM
4        10/7/2021 4:00:00 PM
                ...          
1795    10/31/2021 4:00:00 PM
1796    10/31/2021 4:00:00 PM
1797    10/31/2021 4:00:00 PM
1798    10/31/2021 4:00:00 PM
1799    10/31/2021 4:00:00 PM
Name: date, Length: 1800, dtype: object

In [15]:
from sqlalchemy import create_engine

# Defining our connection variables
username = 'kdyjpzwsyqrqpb' # replace with your username
password = '6b610da9417e361c2af3be1fd0c179061b5b258f0e6f7ff9e1da464ba221e46c' # replace with your password
ipaddress = 'ec2-35-174-35-242.compute-1.amazonaws.com' # change this to your db’s IP address is if not Panoply
port = 5432 # this is the standard port for Postgres, but change it to your port if needed
dbname = 'd2ke5vkurjkusr' # change this to...you guessed it...the name of your db

# A long string that contains the necessary Postgres login information
postgres_str = f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
          
# Create the connection
cnx = create_engine(postgres_str)

In [16]:
table_nm = 'detail'
df_bal.to_sql(table_nm, con=cnx, index=False, if_exists='append')

### 자산군별 비중(general 테이블) INSERT

In [17]:
df_bal = pd.read_csv(access_path+bal_file, parse_dates=['date'])

In [18]:
df_bal_gby = df_bal.loc[:, ['date', 'userid', 'value', 'asset_class']].sort_values(
    ['date', 'userid'],
    ascending=True).groupby(['date', 'userid', 'asset_class'
                             ]).sum().reset_index(drop=False)

In [19]:
df_bal_gby = df_bal_gby.assign(wt=df_bal_gby.groupby(['date', 'userid'])['value'].transform(lambda x: x/x.sum()))

In [20]:
df_bal_gby.date = df_bal_gby['date'].dt.strftime('%#m/%#d/%Y 4:%M:%S PM')
# df_bal_gby.date = df_bal_gby['date'].dt.strftime('%-m/%-d/%Y 4:%M:%S PM')

In [21]:
df_bal_gby

,date,userid,asset_class,value,wt
0,10/7/2021 4:00:00 PM,A001,Alternative,12060.0,0.017014
1,10/7/2021 4:00:00 PM,A001,Cash,6210.0,0.008761
2,10/7/2021 4:00:00 PM,A001,Equity,59122.0,0.083410
3,10/7/2021 4:00:00 PM,A001,Fixed Income,631420.0,0.890814
4,10/7/2021 4:00:00 PM,A002,Alternative,24120.0,0.024251
...,...,...,...,...,...
895,10/31/2021 4:00:00 PM,A008,Fixed Income,57210.0,0.058434
896,10/31/2021 4:00:00 PM,A009,Alternative,300875.0,0.237141
897,10/31/2021 4:00:00 PM,A009,Cash,7460.0,0.005880
898,10/31/2021 4:00:00 PM,A009,Equity,903215.0,0.711888


In [22]:
table_nm = 'general'
df_bal_gby.to_sql(table_nm, con=cnx, index=False, if_exists='append')